In [1]:
import pandas as pd

In [2]:
from wikidata.client import WikiDataClient

In [3]:
countries_query =  \
"""
SELECT ?item ?itemLabel ?population
WHERE
{
  ?item wdt:P31 wd:Q6256.
  ?item wdt:P1082 ?population.
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} 
ORDER BY DESC(?population)
"""

In [4]:
query = """
SELECT ?itemLabel ?item ?propertyLabel  ?lowerBound ?value ?upperBound ?unitLabel ?statement
WHERE
{{
  <{}>  ?predicate                  ?statement.
  ?item          ?predicate                  ?statement.
  ?property      wikibase:claim              ?predicate.
  ?property      wikibase:statementValue     ?valuePred.
  ?statement     ?valuePred                  ?valueNode.
  ?valueNode     wikibase:quantityAmount     ?value.
  ?valueNode     wikibase:quantityUnit       ?unit.
  ?statement     wikibase:rank               ?rank.

  FILTER(?rank != wikibase:DeprecatedRank)

  OPTIONAL {{
          ?valueNode  wikibase:quantityLowerBound ?lowerBound.
          ?valueNode  wikibase:quantityUpperBound ?upperBound.
  }}

  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
}}
"""

In [5]:
qual_query = """
SELECT ?propLabel ?value
WHERE 
{{
  <{}> ?pred ?value.
  ?prop wikibase:qualifier ?pred.
  
  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
}}
"""

In [6]:
df = WikiDataClient.query(countries_query)
df

,item,population,itemLabel
0,http://www.wikidata.org/entity/Q148,1409517397,People's Republic of China
1,http://www.wikidata.org/entity/Q668,1349217956,India
2,http://www.wikidata.org/entity/Q30,325145963,United States of America
3,http://www.wikidata.org/entity/Q252,263991379,Indonesia
4,http://www.wikidata.org/entity/Q155,208494900,Brazil
...,...,...,...
156,http://www.wikidata.org/entity/Q695,20918,Palau
157,http://www.wikidata.org/entity/Q697,13650,Nauru
158,http://www.wikidata.org/entity/Q672,9876,Tuvalu
159,http://www.wikidata.org/entity/Q34020,1612,Niue


In [7]:
df[df.columns.intersection(['item','population'])]

,item,population
0,http://www.wikidata.org/entity/Q148,1409517397
1,http://www.wikidata.org/entity/Q668,1349217956
2,http://www.wikidata.org/entity/Q30,325145963
3,http://www.wikidata.org/entity/Q252,263991379
4,http://www.wikidata.org/entity/Q155,208494900
...,...,...
156,http://www.wikidata.org/entity/Q695,20918
157,http://www.wikidata.org/entity/Q697,13650
158,http://www.wikidata.org/entity/Q672,9876
159,http://www.wikidata.org/entity/Q34020,1612


In [8]:
df2 = WikiDataClient.query(query.format(df.item.iloc[0]))
df2

,lowerBound,upperBound,statement,item,value,itemLabel,propertyLabel,unitLabel
0,6.6,6.8,http://www.wikidata.org/entity/statement/Q148-...,http://www.wikidata.org/entity/Q148,6.700000e+00,People's Republic of China,real gross domestic product growth rate,percent
1,9596960.0,9596962.0,http://www.wikidata.org/entity/statement/Q148-...,http://www.wikidata.org/entity/Q148,9.596961e+06,People's Republic of China,area,square kilometre
2,4.0,6.0,http://www.wikidata.org/entity/statement/Q148-...,http://www.wikidata.org/entity/Q148,5.000000e+00,People's Republic of China,unemployment rate,percent
3,2.0,2.2,http://www.wikidata.org/entity/statement/Q148-...,http://www.wikidata.org/entity/Q148,2.100000e+00,People's Republic of China,inflation rate,percent
4,NaN,NaN,http://www.wikidata.org/entity/statement/Q148-...,http://www.wikidata.org/entity/Q148,7.523600e+01,People's Republic of China,life expectancy,year
...,...,...,...,...,...,...,...,...
302,NaN,NaN,http://www.wikidata.org/entity/statement/Q148-...,http://www.wikidata.org/entity/Q148,1.040000e+02,People's Republic of China,nominal GDP per capita,United States dollar
303,NaN,NaN,http://www.wikidata.org/entity/statement/Q148-...,http://www.wikidata.org/entity/Q148,5.880000e-01,People's Republic of China,Human Development Index,1
304,NaN,NaN,http://www.wikidata.org/entity/statement/Q148-...,http://www.wikidata.org/entity/Q148,1.106467e+13,People's Republic of China,nominal GDP,United States dollar
305,NaN,NaN,http://www.wikidata.org/entity/statement/Q148-...,http://www.wikidata.org/entity/Q148,2.306867e+11,People's Republic of China,nominal GDP,United States dollar


In [9]:
df2.statement.iloc[300]

'http://www.wikidata.org/entity/statement/Q148-DFA3F48A-47B0-4074-9CC2-A2FBFB4C5525'

In [10]:
sts = WikiDataClient.query(qual_query.format(df2.statement.iloc[300]))
sts

,value,propLabel
0,2016-01-01 00:00:00+00:00,point in time


In [12]:
sts.value[0]

Timestamp('2016-01-01 00:00:00+0000', tz='UTC')

In [27]:
', '.join((sts['propLabel']+': '+sts['value']).values)

'point in time: 2016-01-01T00:00:00Z'

In [55]:
print(query.format(df.item.iloc[0]))


SELECT ?itemLabel ?item ?propertyLabel  ?lowerBound ?value ?upperBound ?unitLabel ?statement
WHERE
{
  <http://www.wikidata.org/entity/Q148>  ?predicate                  ?statement.
  ?item          ?predicate                  ?statement.
  ?property      wikibase:claim              ?predicate.
  ?property      wikibase:statementValue     ?valuePred.
  ?statement     ?valuePred                  ?valueNode.
  ?valueNode     wikibase:quantityAmount     ?value.
  ?valueNode     wikibase:quantityUnit       ?unit.
  ?statement     wikibase:rank               ?rank.

  FILTER(?rank != wikibase:DeprecatedRank)

  OPTIONAL {
          ?valueNode  wikibase:quantityLowerBound ?lowerBound.
          ?valueNode  wikibase:quantityUpperBound ?upperBound.
  }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

